# 2. Minio as an S3 service (practical)

***

**ELMI 2021 NGFF Workshop**, 25 June 2021

***



## Summary

* 2.1. Software versions
* 2.2. Notebook reminders
* 2.3. Uploading and downloading to S3
  * 2.3.1 Minio Client (mc)
  * 2.3.2 Connections
  * 2.3.3 Adding your own
  * 2.3.4 Your first download

***

## 2.1. Software versions used for this workshop:


### 2.1.1 Running now

* jupyter                   1.0.0 ([external guide](https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/))
* conda 4.8.4 ([external guide](https://docs.conda.io/projects/conda/en/latest/user-guide/index.html))

### 2.1.2 Core tools

* **minio-client                     2020.11.17** ([install locally](https://docs.min.io/docs/minio-client-complete-guide.html))

### 2.1.3 Other

   * awscli                    1.18.219

***

## 2.2. Notebook reminders

This notebook is somewhat unusual in that we use a lot of command-line tools. Each of the lines beginning with a exclamation mark (`!`) is run in a terminal.

For example, you can use standard bash commands like `pwd` to see what directory you are in. When using mybinder, this will start with `/home/jovyan`. If you are interested in _why_, see [What is a Jovyan?](https://jupyter.readthedocs.io/en/latest/community/content-community.html#what-is-a-jovyan)


In [1]:
!pwd

/opt/elmi2021


The `binder/` directory contains a Conda [environment.yml](https://github.com/joshmoore/NGFF-GBI-2021-Workshop/blob/main/binder/environment.yml) file. You can use it to download all the tools we are using here.

In [3]:
!ls binder/

apt.txt         environment.yml


In [4]:
!cat binder/environment.yml

channels:
  - ome
  - conda-forge
  - joshmoore
  - defaults
dependencies:
  - python == 3.9.0
  - napari
  - ipython
  - wheel
  - requests
  - s3fs
  - scikit-image
  - scipy
  - xarray
  - zarr >= 2.4.0
  - bioformats2raw
  - omero-py
  - go-mc
  - pip
  - pip:
      - PyQT5==5.13.1
      - ome-zarr
      - awscli
      - omero-cli-zarr


You can perform those actions on your own system _without_ a Jupyter notebook. For that, you will need to install the correct tools like `bioformats2raw` or run everything via `repo2docker`.

See instructions under https://github.com/joshmoore/NGFF-ELMI-2021-Workshop/blob/main/README.md
 
*** 

## 2.3.1 Minio client

There are a number of different types of cloud storage and there are a number of tools that you can use to access your cloud storage, but here we're going to focus on a single one `mc`.

`mc` is provided by the minio project and is described as "a modern alternative to UNIX commands like ls, cat, cp, mirror, diff, find etc." The quickstart guide can be found under https://docs.minio.io/docs/minio-client-quickstart-guide.html For our purposes we'll focus on how to use it to upload and manage data in S3.

In [5]:
!mc

NAME:
  mc - MinIO Client for cloud storage and filesystems.

USAGE:
  mc [FLAGS] COMMAND [COMMAND FLAGS | -h] [ARGUMENTS...]

COMMANDS:
  alias      set, remove and list aliases in configuration file
  ls         list buckets and objects
  mb         make a bucket
  rb         remove a bucket
  cp         copy objects
  mirror     synchronize object(s) to a remote site
  cat        display object contents
  head       display first 'n' lines of an object
  pipe       stream STDIN to an object
  share      generate URL for temporary access to an object
  find       search for objects
  sql        run sql queries on objects
  stat       show object metadata
  mv         move objects
  tree       list buckets and objects in a tree format
  du         summarize disk usage recursively
  retention  set retention for object(s)
  legalhold  manage legal hold for object(s)
  diff       list differences in object name, size, and date between two buckets
  rm         remove objects
  version    

## 2.3.2 Connections

The minio project provides a safe space for you to learn about S3: https://play.minio.io:9000/minio/ Here we've used the `mc` command to find the access information:

 * **"AccessKey"** is basically a user name.
 * **"SecretKey"** is basically a password. 
 * The URL is our **"endpoint"**, which differentiates it from the S3 servers provided by Amazon.

You can log in to the webpage and explore what the many other users have upload at https://play.minio.io:9000/minio/

The other two important concepts are:
 * **"buckets"** which is roughly like a shared namespace with permissions
 * and **"keys"** which will get to in a second.

In [8]:
!mc config host list play

play
  URL       : https://play.min.io
  AccessKey : Q3AM3UQ867SPQQA43P2F
  SecretKey : zuf+tfteSlswRu7BJ86wekitnifILbZam1KYY3TG
  API       : S3v4
  Path      : auto



## 2.3.3 Adding your own

But EMBL has kindly provided us a bucket for this session which we need to connect to. Please enter the password you were sent below:

In [13]:
# Python block for capturing a password
import getpass
s3_pass = getpass.getpass()

········


In [14]:
!mc config host add gbi https://s3.embl.de bioim-user $s3_pass

Added `gbi` successfully.


In [15]:
!mc ls gbi/bioim

[2021-01-28 20:38:06 CET] 3.1KiB 404.html
[2021-01-28 20:38:09 CET]  19KiB index.html
[2021-01-21 18:26:48 CET]  32KiB s3-browser.html
[2021-01-29 12:58:58 CET]     0B _next/
[2021-01-29 12:58:58 CET]     0B idr0023/
[2021-01-29 12:58:58 CET]     0B idr0062-tiffs/
[2021-01-29 12:58:58 CET]     0B idr0062-zarrs/
[2021-01-29 12:58:58 CET]     0B josh/


In [16]:
!mc ls gbi/bioim/idr0062-tiffs/

[2021-01-28 17:46:35 CET]  68MiB B1_C1.tif
[2021-01-28 17:46:35 CET]  67MiB B1_C1_Manual.tif
[2021-01-28 17:46:35 CET]  66MiB B1_C2.tif
[2021-01-28 17:46:35 CET]  66MiB B1_C2_Manual.tif
[2021-01-28 17:46:35 CET] 120MiB B2_C1.tif
[2021-01-28 17:46:35 CET] 120MiB B2_C1_Manual.tif
[2021-01-28 17:46:35 CET]  42MiB B2_C2.tif
[2021-01-28 17:46:35 CET]  42MiB B2_C2_Manual.tif
[2021-01-28 17:46:35 CET]  84MiB B3.tif
[2021-01-28 17:46:35 CET]  83MiB B3_Manual.tif
[2021-01-28 17:46:35 CET]  49MiB B4_C1.tif
[2021-01-28 17:46:35 CET]  49MiB B4_C1_Manual.tif
[2021-01-28 17:46:35 CET]  79MiB B4_C2.tif
[2021-01-28 17:46:35 CET]  78MiB B4_C2_Manual.tif
[2021-01-28 17:46:35 CET]  53MiB B4_C3.tif
[2021-01-28 17:46:35 CET]  52MiB B4_C3_Manual.tif
[2021-01-28 17:46:35 CET]  47MiB B5_C1.tif
[2021-01-28 17:46:35 CET]  47MiB B5_C1_Manual.tif
[2021-01-28 17:46:35 CET]  44MiB B5_C2.tif
[2021-01-28 17:46:35 CET]  44MiB B5_C2_Manual.tif
[2021-01-28 17:46:35 CET]  44MiB B5_C3.tif
[2021-01-28 17:46:35 CET]  44MiB 

In [17]:
!mc ls gbi/bioim/idr0062-zarrs/

[2021-01-29 12:59:59 CET]     0B 6001237.zarr/
[2021-01-29 12:59:59 CET]     0B 6001238.zarr/
[2021-01-29 12:59:59 CET]     0B 6001239.zarr/
[2021-01-29 12:59:59 CET]     0B 6001240.zarr/
[2021-01-29 12:59:59 CET]     0B 6001241.zarr/
[2021-01-29 12:59:59 CET]     0B 6001242.zarr/
[2021-01-29 12:59:59 CET]     0B 6001243.zarr/
[2021-01-29 12:59:59 CET]     0B 6001244.zarr/
[2021-01-29 12:59:59 CET]     0B 6001245.zarr/
[2021-01-29 12:59:59 CET]     0B 6001246.zarr/
[2021-01-29 12:59:59 CET]     0B 6001247.zarr/
[2021-01-29 12:59:59 CET]     0B 6001248.zarr/
[2021-01-29 12:59:59 CET]     0B 6001249.zarr/
[2021-01-29 12:59:59 CET]     0B 6001250.zarr/
[2021-01-29 12:59:59 CET]     0B 6001251.zarr/
[2021-01-29 12:59:59 CET]     0B 6001252.zarr/
[2021-01-29 12:59:59 CET]     0B 6001253.zarr/
[2021-01-29 12:59:59 CET]     0B 6001254.zarr/
[2021-01-29 12:59:59 CET]     0B 6001255.zarr/
[2021-01-29 12:59:59 CET]     0B 6001256.zarr/
[2021-01-29 12:59:59 CET]     0B 6001257.zarr/
[2021-01-29 1

In [18]:
!mc ls gbi/bioim/idr0023/

[2021-01-28 10:22:33 CET]    44B README.txt
[2021-01-28 10:21:53 CET] 2.0MiB trans_norm.tif
[2021-01-29 13:00:27 CET]     0B trans_norm.ome.zarr/


## 2.3.4 Your first download

Now you can see list the contents of the bucket and see that it's there. Note that since other people are working on the same bucket, it may look slightly different each time you run this command.

In [22]:
!mc cp gbi/bioim/idr0023/trans_norm.tif julia.tif

..._norm.tif:  2.04 MiB / 2.04 MiB  ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓  236.18 KiB/s 8s

In [20]:
!ls -ltrah /tmp/trans_norm.tif

-rw-r--r--  1 jamoore  wheel   2.0M Jan 29 13:00 /tmp/trans_norm.tif


### License
Copyright (C) 2021 University of Dundee. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.